# Constructing a Pipeline of Functions

<div class="alert alert-block alert-info">
<b>Warning:</b>
This notebook depends on the Parquet files generated by the notebook <b>01 Preparing the Data</b>. Make sure to run all cells in that notebook before executing this one.
    
In particular, the files required are:
<ul>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet</tt></li>
    <li><tt>ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet</tt></li>
</ul>
</div>

This notebook follows the analysis set up in the notebooks **01 Preparing the Data** and **02 Comparing Data Sources**. The pipeline we lay out here was originally developed in a notebook from the [Bodo-examples](https://github.com/Bodo-inc/Bodo-examples) GitHub repository. [That notebook](https://github.com/Bodo-inc/Bodo-examples/blob/master/notebooks/parking/nyc-parking-tickets.ipynb) provides deeper discussion of the individual functions.

Our purpose here is to emphasize use patterns and how those impact run times. It's enough to know here that we are taking a function similar to `load_parking_tickets` from the last notebook and then executing a sequence of transformations on the resulting DataFrame.

In a workflow you might see used in data exploration with a Jupyter notebook, this notebook defines functions and then executes them to compute results. The results of intermediate computations are returned from function calls as objects in the Python interpreter namespace. The objects can be inspected there before continuing to the next portion of the analysis.

We begin by initializing the IPyParallel engines as ususal.

In [1]:
# This cell does not need to be run on the Bodo.ai platform and should be skipped
import ipyparallel as ipp, psutil
n_proc = min(psutil.cpu_count(logical=False), 8)
rc = ipp.Cluster(engines='mpi', n=n_proc).start_and_connect_sync(activate=True)

Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|██████████| 4/4 [00:05<00:00,  1.49s/engine]


<div class="alert alert-block alert-info">
<b>Reminder:</b>

Having launched IPyParallel, the `px` magics ensure that code is executed on all engines.
    
1. The `%px` line magic executes a single line of code an all engines.
2. The `%%px` cell magic—placed as the first line of a cell—ensures all lines in a given cell are executed on all engines.
3. Lines executed outside of the context of a `px` line or cell magic execute in the namespace of the default Jupyter kernel.

Consult the [IPyParallel documentation](https://ipyparallel.readthedocs.io/en/latest/) for more information about setting up a local cluster. [Bodo's documentation](https://docs.bodo.ai/2022.2/bodo_parallelism/bodo_parallelism_basics/) provides more information about the parallel execution model.
</div>

In [2]:
%%px

import pandas as pd, numpy as np
import bodo, time

%px: 100%|██████████| 4/4 [00:01<00:00,  2.85tasks/s]


## Setting up the Pipeline

The following sequence of cells define functions to use in our pipeline.

+ Notice that the functions all return a string with timing information as well as one or more DataFrames. This is strictly for convenient diagnostics.
+ Remember, the code in all these cells execute on all engines by virtue of the `%%px` cell magic.
+ The cell immediately following each function definition checks that the function works as intended. This is not rigorous testing, but it is a typical development workflow in building data pipelines.
+ The definition of the function `load_violation_precincts_codes` is decorated using `@bodo.jit(distributed=False)`. Using the option `distributed=False` means that these small DataFrames are *replicated* onto all engines. For small data like this that is needed on all engines, replicating the data is not punitive and is in fact useful.

In [3]:
%%px

@bodo.jit
def load_parking_tickets():
    """
    Load data and aggregate by day, violation type, and police precinct.
    """
    start = time.time()
    groupby_cols = ['Issue Date','Violation County','Violation Precinct','Violation Code']
    
    DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2016_segmented.parquet'
    year_2016_df = pd.read_parquet(DATA_SRC)
    year_2016_df = year_2016_df.groupby(groupby_cols, as_index=False)['Summons Number'].count()

    DATA_SRC = 'ParkingData/Parking_Violations_Issued_-_Fiscal_Year_2017_segmented.parquet'
    year_2017_df = pd.read_parquet(DATA_SRC)
    year_2017_df = year_2017_df.groupby(groupby_cols, as_index=False)['Summons Number'].count()
    
    # concatenate all dataframes into one dataframe
    many_year_df = pd.concat([year_2016_df, year_2017_df])
    end = time.time()
    timing_str = f"\n{'Reading Time:':<40}{end - start:8.3f} sec"
    return many_year_df, timing_str

In [4]:
%%px
# Verify that load_parking_tickets works as intended
main_df, output = load_parking_tickets()

# Examine output from load_parking_tickets to ensure it makes sense
if bodo.get_rank() == 0:
    display(main_df.head())
    print(output)

%px:   0%|          | 0/4 [00:18<?, ?tasks/s]

[output:0]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number
0,2015-07-09,K,94.0,70,11
1,2015-07-09,K,94.0,21,134
2,2015-07-09,K,84.0,21,53
3,2015-07-09,K,84.0,37,76
4,2015-07-09,K,90.0,21,134


[stdout:0] 
Reading Time:                              3.687 sec


%px: 100%|██████████| 4/4 [00:18<00:00,  4.53s/tasks]


In [5]:
%%px

@bodo.jit(distributed=False)
def load_violation_precincts_codes():
    """
    Load violation codes and precincts information.
    """
    start = time.time()
    violation_codes = pd.read_csv("ParkingData/DOF_Parking_Violation_Codes.csv")
    violation_codes.columns = ['Violation Code','Definition','manhattan_96_and_below','all_other_areas']
    nyc_precincts_df = pd.read_csv("ParkingData/nyc_precincts.csv", index_col='index')
    end = time.time()
    timing_str = f"\n{'Violation and precincts load Time:':<40}{end - start:8.3f} sec"
    return violation_codes, nyc_precincts_df, timing_str

In [6]:
%%px
# Verify that load_violation_precincts_codes works as intended
violation_codes, nyc_precincts_df, output = load_violation_precincts_codes()
if bodo.get_rank()==0:
    display(violation_codes.head())
    display(nyc_precincts_df.head())
    print(output)

%px:   0%|          | 0/4 [00:03<?, ?tasks/s]

[output:0]

,Violation Code,Definition,manhattan_96_and_below,all_other_areas
0,10,"Stopping, standing or parking where a sign, st...",115,115
1,11,Hotel Loading/Unloading: Standing or parking w...,115,115
2,12,Snow Emergency: Standing or parking where stan...,95,95
3,13,Taxi Stand: Standing or parking where standing...,115,115
4,14,General No Standing: Standing or parking where...,115,115


[output:0]

,Violation Precinct
index,
0,1
1,5
2,6
3,7
4,9


[stdout:0] 
Violation and precincts load Time:         0.115 sec


%px: 100%|██████████| 4/4 [00:03<00:00,  1.17tasks/s]


In [7]:
%%px

@bodo.jit
def elim_code_36(main_df):
    """
    Remove undefined violations (code 36)
    """
    start = time.time()
    main_df = main_df[main_df['Violation Code']!=36].sort_values('Summons Number',ascending=False)
    end = time.time()
    timing_str = f"\n{'Eliminate undefined violations time:':<40}{end - start:8.3f} sec"
    return main_df, timing_str

In [8]:
%%px
# Verify that elim_code_36 works as intended
main_df, output = elim_code_36(main_df)
# Examine output from elim_code_36 to ensure it makes sense
if bodo.get_rank() == 0:
    display(main_df.head())
    print(output)

%px:   0%|          | 0/4 [00:01<?, ?tasks/s]

[output:0]

%px:  50%|█████     | 2/4 [00:01<00:00, 19.66tasks/s]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number
479992,2015-07-11,nan,0.0,7,2365
4588,2015-06-13,nan,0.0,7,2327
337548,2015-07-25,nan,0.0,7,2285
479997,2015-06-24,nan,0.0,7,2204
162627,2015-06-14,nan,0.0,7,2182


[stdout:0] 
Eliminate undefined violations time:       0.190 sec


%px: 100%|██████████| 4/4 [00:01<00:00,  2.28tasks/s]


In [9]:
%%px

@bodo.jit
def remove_outliers(main_df):
    """
    Delete entries that have dates outside our dataset dates
    """
    start = time.time()
    main_df = main_df[(main_df['Issue Date'] >= '2016-01-01') & (main_df['Issue Date'] <= '2017-12-31')]
    end = time.time()
    timing_str = f"\n{'Remove outliers time:':<40}{end - start:8.3f} sec"
    return main_df, timing_str

In [10]:
%%px
# Verify that remove_outliers works as intended
main_df, output = remove_outliers(main_df)
# Examine output from remove_outliers to ensure it makes sense
if bodo.get_rank() == 0:
    display(main_df.head())
    print(output)

%px:  25%|██▌       | 1/4 [00:02<00:00,  9.84tasks/s]

[output:0]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number
282629,2016-04-17,nan,0.0,7,1661
441190,2016-04-16,nan,0.0,7,1603
282589,2016-04-10,nan,0.0,7,1589
611710,2016-04-23,nan,0.0,7,1586
584719,2016-03-12,nan,0.0,7,1545


[stdout:0] 
Remove outliers time:                      0.030 sec


%px: 100%|██████████| 4/4 [00:02<00:00,  1.72tasks/s]


In [11]:
%%px

@bodo.jit
def merge_violation_code(main_df, violation_codes):
    """
    Merge violation information in the main_df
    """
    start = time.time()
    # left join main_df and violation_codes df so that there's more info on violation in main_df
    main_df = pd.merge(main_df, violation_codes, on='Violation Code', how='left')
    # cast precincts as integers from floats (inadvertent type change by merge)
    main_df['Violation Precinct'] = main_df['Violation Precinct'].astype(int)
    end = time.time()
    timing_str = f"\n{'Merge time:':<40}{end - start:8.3f} sec"
    return main_df, timing_str

In [12]:
%%px
# Verify that merge_violation_code works as intended
main_w_violation, output = merge_violation_code(main_df, violation_codes)
# Examine output from merge_violation_code to ensure it makes sense
if bodo.get_rank() == 0:
    display(main_w_violation.head())
    print(output)

%px:  25%|██▌       | 1/4 [00:01<00:00,  9.88tasks/s]

[output:0]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number,Definition,manhattan_96_and_below,all_other_areas
0,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50
1,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50
2,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50
3,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50
4,2016-04-16,nan,0,7,1603,Vehicles photographed going through a red ligh...,50,50


[stdout:0] 
Merge time:                                0.440 sec


%px: 100%|██████████| 4/4 [00:02<00:00,  1.63tasks/s]


In [13]:
%%px
@bodo.jit
def calculate_total_summons(main_df):
    """
    Calculate the total summonses in dollars for a violation in a precinct on a day
    """
    start = time.time()
    # create column for portion of precinct 96th st. and below
    n = len(main_df)
    portion_manhattan_96_and_below = np.empty(n, np.int64)
    # NOTE: To run Pandas, use this loop.
    # for i in range(n):
    for i in bodo.prange(n):
        x = main_df['Violation Precinct'].iat[i]
        if x < 22 or x == 23:
            portion_manhattan_96_and_below[i] = 1.0
        elif x == 22:
            portion_manhattan_96_and_below[i] = 0.75
        elif x == 24:
            portion_manhattan_96_and_below[i] = 0.5
        else: #other
            portion_manhattan_96_and_below[i] = 0
    main_df["portion_manhattan_96_and_below"] = portion_manhattan_96_and_below

    # create column for average dollar amount of summons based on location
    main_df['average_summons_amount'] = (main_df['portion_manhattan_96_and_below'] * main_df['manhattan_96_and_below']
                                     + (1 - main_df['portion_manhattan_96_and_below']) * main_df['all_other_areas'])

    # get total summons dollars by multiplying average dollar amount by number of summons given
    main_df['total_summons_dollars'] = main_df['Summons Number'] * main_df['average_summons_amount']
    main_df = main_df.sort_values(by=['total_summons_dollars'], ascending=False)
    end = time.time()
    timing_str = f"\n{'Calculate Total Summons Time:':<40}{end - start:8.3f} sec"
    return main_df, timing_str

In [14]:
%%px
# Verify that calculate_total_summons works as intended
total_summons, output = calculate_total_summons(main_w_violation)
# Examine output from calculate_total_summons to ensure it makes sense
if bodo.get_rank() == 0:
    display(total_summons.head())
    print(output)

%px:  25%|██▌       | 1/4 [00:18<00:00,  9.59tasks/s]

[output:0]

,Issue Date,Violation County,Violation Precinct,Violation Code,Summons Number,Definition,manhattan_96_and_below,all_other_areas,portion_manhattan_96_and_below,average_summons_amount,total_summons_dollars
0,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50,1,50,83050
1,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50,1,50,83050
2,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50,1,50,83050
3,2016-04-17,nan,0,7,1661,Vehicles photographed going through a red ligh...,50,50,1,50,83050
4,2016-04-16,nan,0,7,1603,Vehicles photographed going through a red ligh...,50,50,1,50,80150


[stdout:0] 
Calculate Total Summons Time:              6.486 sec


%px: 100%|██████████| 4/4 [00:19<00:00,  4.95s/tasks]


In [15]:
%%px
@bodo.jit
def aggregate(main_df):
    '''function that aggregates and filters data
    e.g. total violations by precinct
    '''
    start = time.time()
    filtered_dataset = main_df[['Violation Precinct','Summons Number', 'total_summons_dollars']]
    precinct_offenses_df = filtered_dataset.groupby(by=['Violation Precinct']).sum().reset_index().fillna(0)
    end = time.time()
    timing_str = f"\n{'Aggregate code time:':<40}{end - start:8.3f} sec"
    return precinct_offenses_df, timing_str

In [16]:
%%px
# Verify that aggregate works as intended
precinct_offenses_df, output = aggregate(total_summons)
# Examine output from aggregate to ensure it makes sense
if bodo.get_rank() == 0:
    display(precinct_offenses_df.head())
    print(output)

%px:   0%|          | 0/4 [00:07<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,3198595,279486980
1,104,723902,47944840
2,115,1007961,65570600
3,43,662495,46015940
4,10,760948,70826700


[stdout:0] 
Aggregate code time:                       0.054 sec


%px: 100%|██████████| 4/4 [00:07<00:00,  1.98s/tasks]


---------------------

## Wrapping the Pipeline in a Function

We could keep this notebook as a template computation. Whenever a new dataset of the same form is to be analyzed, we can modify the filename hard-coded into the function `load_parking_tickets` and execute this specific sequence of cells again.

The cell below puts the logic from the preceding sequence of cells into a single function `run_pipeline`.

In [17]:
%%px
# Put all the functions in the pipeline into a single function
def run_pipeline():
    start = time.time()
    
    main_df, out1 = load_parking_tickets()
    violation_codes, nyc_precincts_df, out2 = load_violation_precincts_codes()
    main_df, out3 = elim_code_36(main_df)
    main_df, out4 = remove_outliers(main_df)
    main_w_violation, out5 = merge_violation_code(main_df, violation_codes)
    total_summons, out6 = calculate_total_summons(main_w_violation)
    precinct_offenses_df, out7 = aggregate(total_summons)
    
    end = time.time()
    out8 = f"\n{52*'='}\n{'Execution time (run_pipeline):':<40}{end-start:8.3f} sec"
    output_str = ''.join(['\n', out1, out2, out3, out4, out5, out6, out7, out8, '\n'])
    return precinct_offenses_df, output_str

We execute the `run_pipeline` function here (using also the `%%time` cell magic to return the elapsed time for executing all lines in this cell).

In [18]:
%%time
%%px
# Verify that run_pipeline works as intended
result, output = run_pipeline()
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [00:17<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,3198595,279486980
1,104,723902,47944840
2,115,1007961,65570600
3,43,662495,46015940
4,10,760948,70826700


[stdout:0] (283, 3)


Reading Time:                              6.689 sec
Violation and precincts load Time:         0.054 sec
Eliminate undefined violations time:       0.323 sec
Remove outliers time:                      0.051 sec
Merge time:                                1.290 sec
Calculate Total Summons Time:              6.278 sec
Aggregate code time:                       0.388 sec
Execution time (run_pipeline):            16.653 sec



%px: 100%|██████████| 4/4 [00:17<00:00,  4.43s/tasks]
CPU times: user 514 ms, sys: 89 ms, total: 603 ms
Wall time: 19.7 s


Another subtlety to observe is that the timings inside the functions definitions only measure run-time execution. That is, whatever overhead costs are involved in *compiling* functions are not accounted for in these performance meatures. In this case, the observed discrepancy between the `Wall time` (as returned by `%%time`) and the `Execution time` (as returned by `run_pipeline`) is about 3 seconds (YMMV). As we've already *executed* the functions in the pipeline, those compiled results have been cached. So the overhead here is the time required to compile the function `run_pipeline`.

If we re-execute the cell again (copied below), this compilation overhead effectively vanishes.

In [19]:
%%time
%%px
# Verify that run_pipeline works as intended.
# Executing this a second time requires no compilation.
# The discrepancy between "Wall time" and "Full Run" 
result, output = run_pipeline()
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [00:07<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,3198595,279486980
1,104,723902,47944840
2,115,1007961,65570600
3,43,662495,46015940
4,10,760948,70826700


[stdout:0] (283, 3)


Reading Time:                              3.333 sec
Violation and precincts load Time:         0.018 sec
Eliminate undefined violations time:       0.198 sec
Remove outliers time:                      0.023 sec
Merge time:                                0.424 sec
Calculate Total Summons Time:              3.655 sec
Aggregate code time:                       0.446 sec
Execution time (run_pipeline):             8.881 sec



%px: 100%|██████████| 4/4 [00:07<00:00,  1.77s/tasks]
CPU times: user 168 ms, sys: 25.2 ms, total: 194 ms
Wall time: 9.1 s


There is still a minor difference between `Wall time` and `Execution time`—a few hundred milliseconds. These reflect inevitable communication & set-up overhead that is still much less costly than compilation.

## Boxing & Unboxing

Another potential issue with the pipeline developed above is that data passes in and out of Bodo-compiled functions in several places. The issue is that Bodo and native Python do not always represent data structures in the same way—Bodo represents data using efficient native data structures. This means that passing data from the top-level Python namespace into a Bodo-jitted function involves *'unboxing'* that object from its native Python representation. Similarly, returning an object back to the top-level Python namespace from a Bodo-jitted function requires *'boxing'* that object back into Python's native object representation. These transformations can be costly.

To illustrate this point, let's apply the `bodo.jit` decorator to the function `run_pipeline` from before. The content is identical except for the application of the JIT compiler (and some tailoring of the output).

In [20]:
%%px
# Put all the functions in the pipeline into a single jitted function
@bodo.jit
def run_pipeline_jitted():
    start = time.time()
    
    main_df, out1 = load_parking_tickets()
    violation_codes, nyc_precincts_df, out2 = load_violation_precincts_codes()
    main_df, out3 = elim_code_36(main_df)
    main_df, out4 = remove_outliers(main_df)
    main_w_violation, out5 = merge_violation_code(main_df, violation_codes)
    total_summons, out6 = calculate_total_summons(main_w_violation)
    precinct_offenses_df, out7 = aggregate(total_summons)
    
    end = time.time()
    out8 = f"\n{52*'='}\n{'Execution time (run_pipeline_jitted):':<40}{end-start:8.3f} sec"
    output_str = ''.join(['\n', out1, out2, out3, out4, out5, out6, out7, out8, '\n'])
    return precinct_offenses_df, output_str

Let's execute `run_pipeline_jitted` to see if it works. Remember, the first time this executes, the compilation time is reflected in the discrepancy between the reported `Execution time` and the `Wall time` returned by `%%time`.

In [21]:
%%time
%%px
# Verify that run_pipeline_jitted works as intended
result, output = run_pipeline_jitted()
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [00:10<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,799846,69871745
1,104,181028,11986210
2,115,252045,16392650
3,43,165959,11503985
4,10,190549,17706675


%px:  75%|███████▌  | 3/4 [00:10<00:00, 29.49tasks/s]

[stdout:0] (283, 3)


Reading Time:                              3.509 sec
Violation and precincts load Time:         0.012 sec
Eliminate undefined violations time:       0.155 sec
Remove outliers time:                      0.010 sec
Merge time:                                0.088 sec
Calculate Total Summons Time:              0.417 sec
Aggregate code time:                       0.060 sec
Execution time (run_pipeline_jitted):      4.251 sec



%px: 100%|██████████| 4/4 [00:10<00:00,  2.59s/tasks]
CPU times: user 223 ms, sys: 37.1 ms, total: 260 ms
Wall time: 12.4 s


The differences in the observed `Execution time` results are noticeable (about 4 seconds for `run_pipeline` vs. 10 seconds for `run_pipeline_jitted`). As the functions internal to `run_pipeline_jitted` have also been passed through `bodo.jit`, the penalty for boxing & unboxing Python data structures is removed. Notice this is a *run-time* cost; it will persist in repeated executions (unlike the compilation cost).

We'll repeat the preceding cell here to see.

In [22]:
%%time
%%px
# Verify that run_pipeline_jitted works as intended
result, output = run_pipeline_jitted()
final_result = bodo.gatherv(result)
if bodo.get_rank()==0:
    display(final_result.head())
    print(final_result.shape)
    print(output)

%px:   0%|          | 0/4 [00:02<?, ?tasks/s]

[output:0]

,Violation Precinct,Summons Number,total_summons_dollars
0,19,799846,69871745
1,104,181028,11986210
2,115,252045,16392650
3,43,165959,11503985
4,10,190549,17706675


[stdout:0] (283, 3)


Reading Time:                              3.303 sec
Violation and precincts load Time:         0.012 sec
Eliminate undefined violations time:       0.155 sec
Remove outliers time:                      0.011 sec
Merge time:                                0.082 sec
Calculate Total Summons Time:              0.432 sec
Aggregate code time:                       0.039 sec
Execution time (run_pipeline_jitted):      4.038 sec



%px: 100%|██████████| 4/4 [00:02<00:00,  1.93tasks/s]
CPU times: user 60.2 ms, sys: 14.6 ms, total: 74.8 ms
Wall time: 4.07 s


Again, once `run_pipeline_jitted` has compiled once, the `Wall time` is negligibly more than the `Execution time`. For data this small, the above is adequate and, unless we're running `run_pipeline_jitted` thousands or millions of times, we're probably content at this point.

In [23]:
# To stop the cluster run the following command.
rc.cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 1496, 'identifier': 'ipcontroller-1651428426-wz01-1473'}
Stopping engine(s): 1651428427


In the next notebook, we'll work with this data pipeline again, modifying it to see how it performs as the data size scales.

---------------------